In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Create a function to load and prepare images
def load_and_prep_image(filename, img_shape = 128, scale = True):
    # Read in image
    img = tf.io.read_file(filename)
    # Decode image into tensor
    img = tf.image.decode_jpeg(img, channels = 3)
    # Resize the image
    img = tf.image.resize(img, size = [img_shape, img_shape])
    # Scale? Yes or No
    if scale:
        # rescale the image (get all values between 0 and 1)
        return img/255.
    else:
        return img # don't need to rescale images for EfficientNet models in TensorFlow

In [4]:
# Import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import datetime
import itertools
import random
import os

In [5]:
import tensorflow as tf

# Specify the path where your model is saved
model_path = "/content/drive/MyDrive/Model_train/car_model_embed.h5"  # Change this to the actual path of your saved model

# Load the model
embedding_model = tf.keras.models.load_model(model_path)

# Extract the embedding from the hidden stage
# embedding_model = tf.keras.Model(
#     inputs=loaded_model.input,
#     # outputs=loaded_model.get_layer("global_avg_pool_layer").output
# )

# Display the architecture of the embedding model
embedding_model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2_input (InputLayer  [(None, 128, 128, 3)]     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                            
                                                           

In [6]:
import pandas as pd

df = pd.DataFrame(columns=['path', 'id'])

img_list = os.listdir('/content/drive/MyDrive/cars_img')
for i in range(len(img_list)):
    df.loc[len(df)] = [img_list[i], img_list[i].split('.')[0]]

df = df[df['path'] != '.ipynb_checkpoints']
df['check'] = np.nan
df

,path,id,check
0,21047917-f7f3-4de9-a712-67b9cedc0b89.jpg,21047917-f7f3-4de9-a712-67b9cedc0b89,NaN
1,47fa9c2a-fce9-4daf-9da6-b9ee2c065f75.jpg,47fa9c2a-fce9-4daf-9da6-b9ee2c065f75,NaN
2,11a90ccf-0356-4001-ac71-cfd276e94890.jpg,11a90ccf-0356-4001-ac71-cfd276e94890,NaN
3,7e99741a-7d44-45de-93a7-c16e29091223.jpg,7e99741a-7d44-45de-93a7-c16e29091223,NaN
4,c57f23aa-3806-4044-96d8-efb17465b45c.jpg,c57f23aa-3806-4044-96d8-efb17465b45c,NaN
...,...,...,...
11430,ffcf2b48-514e-4b9e-8a83-65d5ffb243a8.jpg,ffcf2b48-514e-4b9e-8a83-65d5ffb243a8,NaN
11431,4de0a5c2-c035-4a0d-8702-5dcfd9adfb8b.jpg,4de0a5c2-c035-4a0d-8702-5dcfd9adfb8b,NaN
11432,bd4b4f0d-6daa-4b12-8400-19f8725bc86e.jpg,bd4b4f0d-6daa-4b12-8400-19f8725bc86e,NaN
11433,a16dbd13-6487-4982-9b13-242cc0bcebda.jpg,a16dbd13-6487-4982-9b13-242cc0bcebda,NaN


In [7]:
item = df.head(0)
item

,path,id,check


In [ ]:
!pip install faiss-cpu

# Chạy FAISS
from tqdm import tqdm, tqdm_notebook
import numpy as np
import faiss

hidden_dim = 1280
#Create Index
embedded_list = faiss.IndexFlatL2(hidden_dim)


count = 0

for index,row in tqdm_notebook(df.iterrows()):
  try:
    filepath = '/content/drive/MyDrive/cars_img/' + row['path']
    img = load_and_prep_image(filepath, scale = True) # efficientnet
    embedded = embedding_model.predict(tf.expand_dims(img, axis = 0))
    embedded_list.add(embedded)   # Add embedding
    item.loc[len(item)] = row.to_list()

    count +=1

    if count%100==0:
        name_output_file = '/content/drive/MyDrive/Model_train/car.csv'
        item.to_csv(name_output_file)
        name_output_file = '/content/drive/MyDrive/Model_train/car.bin'
        faiss.write_index(embedded_list, name_output_file)
  except:
    print(f'Error {row}')


print(f'DONE')

In [ ]:
def image_grid(imgs, rows, cols):
    w,h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    for i, img in enumerate(imgs): grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:

## Load database vector
index_path = '/content/drive/MyDrive/Model_train/car.bin'
load_jacket_embed = faiss.read_index(index_path)

In [ ]:
#@title Nhập đường dẫn ảnh (url)
from PIL import Image
INPUT_IMAGE_URL = "https://www.mbusa.com/content/dam/mb-nafta/us/future-vehicles/my24/amg-gt-coupe/specialty-hero/2024-AMG-GT-COUPE-CH-DR.jpg" #@param {type:"string"}
variable_name = 4 # @param {type:"slider", min:0, max:10, step:1}
# Tải về
!wget -q -O input.jpg $INPUT_IMAGE_URL


## Xử lý ảnh mới (ảnh muốn tìm kiếm - user input)
img = load_and_prep_image('input.jpg', scale = True) # efficientnet
new_embed = embedding_model.predict(tf.expand_dims(img, axis = 0))

## Tìm kiếm 3 vector tương đồng
f_distances, f_ids = embedded_list.search(new_embed, k=3)
print(f_ids)

retrieved_examples = []
for i in f_ids[0]:
  path = '/content/drive/MyDrive/cars_img/' + item['path'][i]
  print(item['path'][i].split('_')[0])
  anh = Image.open(path)
  retrieved_examples.append(anh)

output_list = [Image.open('input.jpg')]
output_list.extend(retrieved_examples)

image_grid(output_list, 1, len(output_list))